In [2]:
!pip install gradio -U -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.


In [ ]:
!cd ~/codenator-automatic-code-generation-and-execution-using-llm/web_ui/app && python3 webui.py

In [74]:
!docker build --quiet --tag webui .

sha256:9a12e275d24bddd64063f341cb341f55524fd56acb93038ac9e7573aac241399


In [3]:
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                       ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                           
m5.large_i-0d888dfcf3cf3538a *                                             tcp://ip-10-3-1-199.us-west-2.compute.internal:1111   


In [4]:
host_ip1 = "internal-codenator-899847730.us-west-2.elb.amazonaws.com"
host_ip = "ip-10-3-1-199.us-west-2.compute.internal" 

In [75]:
%%bash -s $host_ip -s $host_ip1
echo ${3}
docker run -d -p 8001:8001 -e AWS_REGION=us-west-2 -e AWS_DEFAULT_REGION=us-west-2 --name test-webui webui --host 0.0.0.0 --port 8001 --controller-host ${3} --controller-port 8012 --models-metadata-db model-db --share True --debug True

internal-codenator-899847730.us-west-2.elb.amazonaws.com
7e69fc0e98304d157016ee7e0142d2ca04c9d2bfd12ceee2194638d86bfd6959


In [78]:
!docker logs test-webui

In [40]:
!docker rm test-webui

test-webui


In [73]:
!docker kill test-webui && docker rm test-webui

test-webui
test-webui


In [34]:
import boto3
import json

br_client = boto3.client("bedrock-runtime")

res = ' {\n  "body": "Generate code to connect to SageMaker and print the names of the first three models.", \n  "reasoning": "The code imports the boto3 library to connect to AWS services. It tries to create a SageMaker client, call the list_models API to get a response, slice the first three models from the response and print their names in a for loop. Any exceptions are caught and printed for debugging purposes."\n}'

text = json.loads(res)["body"]

embedding = json.loads(
    br_client.invoke_model(
        body=json.dumps(
            {
                "inputText":text
            }
        ),
        modelId="amazon.titan-embed-text-v1"
    )["body"].read())["embedding"]

print(len(embedding))

1536


In [35]:
!pip install requests_aws4auth

  Using cached requests_aws4auth-1.2.3-py2.py3-none-any.whl (24 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [59]:
import boto3
import requests 
from requests_aws4auth import AWS4Auth

region = 'us-west-2'
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

# Register repository
url = "https://os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com"

payload = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 512
    }
  },
  "mappings": {
    "properties": {
      "task-store-cosine-field": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "parameters": {},
          "space_type": "cosinesimil"
        }
      }
    }
  }
}
headers = {"Content-Type": "application/json"}

r = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)

404



In [60]:
r.

<bound method Response.json of <Response [404]>>

In [37]:
%%bash
curl -v --aws-sigv4 "aws:amz:us-west-2:aoss" -H "Content-Type: application/json" -X PUT https://os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com/task-store-cosine-index
{
  "settings": {
    "index": {
      "knn": true,
      "knn.algo_param.ef_search": 512
    }
  },
  "mappings": {
    "properties": {
      "task-store-cosine-field": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "parameters": {},
          "space_type": "cosinesimil"
        }
      }
    }
  }
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.3.3.166:443...
* Connected to os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com (10.3.3.166) port 443 (#0)
* ALPN: offers h2
* ALPN: offers http/1.1
*  CAfile: /opt/conda/ssl/cacert.pem
*  CApath: none
} [5 bytes data]
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
} [512 bytes data]
* TLSv1.3 (IN), TLS handshake, Server hello (2):
{ [122 bytes data]
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
{ [10 bytes data]
* TLSv1.3 (IN), TLS handshake, Certificate (11):
{ [5067 bytes data]
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
{ [264 bytes data]
* TLSv1.3 (IN), TLS handshake, Finished (20):
{ [36 bytes data]
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
} [1 bytes data]
* TLSv1.3 (OUT), TLS handshake, Finished 

{"status":403,"request-id":"422e3f30-ac63-9497-a7cd-f15dd265081e","error":{"reason":"403 Forbidden","type":"Forbidden"}}


bash: line 3: settings:: command not found
bash: line 4: index:: command not found
bash: line 5: knn:: command not found
bash: line 6: knn.algo_param.ef_search:: command not found
bash: line 8: },: command not found
bash: line 9: mappings:: command not found
bash: line 10: properties:: command not found
bash: line 11: task-store-cosine-field:: command not found
bash: line 12: type:: command not found
bash: line 13: dimension:: command not found
bash: line 14: method:: command not found
bash: line 15: name:: command not found
bash: line 16: engine:: command not found
bash: line 17: parameters:: command not found
bash: line 18: space_type:: command not found
bash: line 19: syntax error near unexpected token `}'
bash: line 19: `        }'


CalledProcessError: Command 'b'curl -v --aws-sigv4 "aws:amz:us-west-2:aoss" -H "Content-Type: application/json" -X PUT https://os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com/task-store-cosine-index\n{\n  "settings": {\n    "index": {\n      "knn": true,\n      "knn.algo_param.ef_search": 512\n    }\n  },\n  "mappings": {\n    "properties": {\n      "task-store-cosine-field": {\n        "type": "knn_vector",\n        "dimension": 1536,\n        "method": {\n          "name": "hnsw",\n          "engine": "nmslib",\n          "parameters": {},\n          "space_type": "cosinesimil"\n        }\n      }\n    }\n  }\n}\n'' returned non-zero exit status 2.

In [14]:
host_ip = "internal-codenator-899847730.us-west-2.elb.amazonaws.com"

In [18]:
%%bash -s $host_ip
curl -X POST ${1}:8010/invoke -d '{"body":{"prompt": "\n<s>[INST] You are a helpful AI assistant that generates code.\\nYou have access to a Python code interpreter environment, which supports you in your tasks.\\nYou respond by generating Python code to answer user instructions.\\nThe code is executed in an interactive shell.\\nThe environment has internet, file system access and access to AWS my account.\\nTo answer instructions about my AWS account, generate Python code.\\nWhen generating code, use the following format:\\n```python\\n<your code>\\n```\\n\\nReport expected output and enclose it within a <output></output> tag.\\nIf you want to call Python and still say something, do only output text above the code block, NOT below.\\nOnly provide at most one code block per message.\\nThe code will be executed automatically.\n[/INST]Thank you for the explanation. I understand I have access to a Python interpreter and can execute code by placing it between ```python tags then return expected output between <output></output> tags. I will use this to assist you with any Python programming tasks. Just let me know what you need help with!\n<s>[INST]Write flask app with debug logs\n[/INST]", "stream": false}, "model_family": "sagemaker", "model_name": "llama2js.jumpstart-dft-meta-textgeneration-llama-2-13b"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1371  100    82  100  1289   1321  20780 --:--:-- --:--:-- --:--:-- 22475


{"error":"An error occurred: Expecting ',' delimiter: line 4 column 21 (char 59)"}

In [29]:
%%bash -s $host_ip
curl -X POST ${1}:8010/invoke -d '{"body":{"prompt": "\n<s>[INST]Hello[/INST]", "stream": false}, "model_family": "sagemaker", "model_name": "1llama2js.jumpstart-dft-meta-textgeneration-llama-2-13b"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   233  100    68  100   165  14055  34105 --:--:-- --:--:-- --:--:-- 58250


{"error":"An error occurred: No module named 'handlers.sagemaker1'"}

In [24]:
a = '{"body":{"prompt": "\\n<s>[INST]Hello[/INST]", "stream": false}, "model_family": "sagemaker", "model_name": "llama2js.jumpstart-dft-meta-textgeneration-llama-2-13b"}'

import json

json.loads(a)

{'body': {'prompt': '\n<s>[INST]Hello[/INST]', 'stream': False},
 'model_family': 'sagemaker',
 'model_name': 'llama2js.jumpstart-dft-meta-textgeneration-llama-2-13b'}

In [5]:
"<exp_out>ex_out_tag▌".rstrip("▌")

'<exp_out>ex_out_tag'

In [18]:
import json

a = json.loads(b'"{\\"error\\": \\"\'generated_text\'\\", \\"stacktrace\\": \\"Traceback (most recent call last):\\\\n  File \\\\\\"/code/app/main.py\\\\\\", line 93, in generate\\\\n    res = conv.send_to_agent(stream)\\\\n  File \\\\\\"/code/app/conversation.py\\\\\\", line 75, in send_to_agent\\\\n    res = self.agent(\\\\n  File \\\\\\"/code/app/utils.py\\\\\\", line 96, in send_req_to_agent\\\\n    return json.loads(ret.text)[\\\\\\"generated_text\\\\\\"]\\\\nKeyError: \'generated_text\'\\\\n\\"}"')

In [20]:
a

'{"error": "\'generated_text\'", "stacktrace": "Traceback (most recent call last):\\n  File \\"/code/app/main.py\\", line 93, in generate\\n    res = conv.send_to_agent(stream)\\n  File \\"/code/app/conversation.py\\", line 75, in send_to_agent\\n    res = self.agent(\\n  File \\"/code/app/utils.py\\", line 96, in send_req_to_agent\\n    return json.loads(ret.text)[\\"generated_text\\"]\\nKeyError: \'generated_text\'\\n"}'

In [24]:
raise Exception("test")

Exception: test

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt

# Get Bitcoin price data for the last year 
btc = yf.Ticker("BTC-USD")
hist = btc.history(period="max")

# Plot the closing price
plt.plot(hist['Close'])

# Add labels and title  
plt.xlabel('Date') 
plt.ylabel('Closing Price (USD)')
plt.title('Bitcoin Price Last Year')

# Save the plot
plt.savefig('btc_price.png') 

# Show the plot
plt.show()